### Собственно что это?

Как говорится не сломаешь не поймешь: используя знания от безусловной генерации я хочу попробовать придумать условную генерацию по метке но только уже свою

In [3]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

import pandas as sns
import numpy as np
import matplotlib.pyplot as plt

import kagglehub
import idx2numpy
import os

In [4]:
def read_mnist():
    path = kagglehub.dataset_download("hojjatk/mnist-dataset")
    print("Путь до данных:", path)

    train_images_path = os.path.join(path, 'train-images.idx3-ubyte')
    train_labels_path = os.path.join(path, 'train-labels.idx1-ubyte')
    test_imaages_path = os.path.join(path, 't10k-images.idx3-ubyte')
    test_labels_path = os.path.join(path, 't10k-labels.idx1-ubyte')
    
    X_train = idx2numpy.convert_from_file(train_images_path)
    y_train = idx2numpy.convert_from_file(train_labels_path)
    X_test = idx2numpy.convert_from_file(test_imaages_path)
    y_test = idx2numpy.convert_from_file(test_labels_path)

    print(f'Загружено тренировочных картинток: {len(X_train)}')
    print(f'Загружено тренировочных меток: {len(y_train)}')
    print(f'Загружено тестовых картинток: {len(X_test)}')
    print(f'Загружено тестовых меток: {len(y_test)}')
    
    return X_train, X_test, y_train, y_test

In [5]:
X_train, X_test, y_train, y_test = read_mnist()

Путь до данных: C:\Users\panika\.cache\kagglehub\datasets\hojjatk\mnist-dataset\versions\1
Загружено тренировочных картинток: 60000
Загружено тренировочных меток: 60000
Загружено тестовых картинток: 10000
Загружено тестовых меток: 10000


In [6]:
def normalize_data(X):
    X_zero_one = X / 255.0
    X_minus_one_one = (X_zero_one - 0.5) * 2.0
    return X_minus_one_one

batch_size = 128
X = torch.tensor(X_train, dtype=torch.float32)
X_norm = normalize_data(X)
X_batch = X_norm.view(-1, 1, 28, 28)
train_loader = DataLoader(TensorDataset(X_batch), batch_size=batch_size, shuffle=True)

In [9]:
timesteps = 1000
beta_start = 0.0001
beta_end = 0.02
betas = torch.linspace(beta_start, beta_end, timesteps)
alphas = 1 - betas
alphas_cumprod = torch.cumprod(alphas, axis=0)

In [7]:
def q_sample(x_start, t, alphas_cumprod, noise):  
    """
    Add noise to x_start
    Args:
        x_start: [Batch, canals, W, H]
        t: [B,]
        alphas_cumprod: array of alphas
        noise: [Batch, canals, W, H] ~ N(0, I)
    """
    sqrt_alphas_cumprod_t = torch.sqrt(alphas_cumprod[t]).view(-1, 1, 1, 1)
    sqrt_one_minus_alphas_cumprod_t = torch.sqrt(1 - alphas_cumprod[t]).view(-1, 1, 1, 1)

    return sqrt_alphas_cumprod_t * x_start + sqrt_one_minus_alphas_cumprod_t * noise

def time_embedding(t, dim):
    """
    n = dim
    t -> {sin(w1*t), cos(w1*t), ... sin(wn*t), cos(wn*t)}
    Args:
        t: [B, ]
        dim: scalar
    """
    idx = torch.arange(0, dim // 2, device=t.device)
    w = 1 / (10000 ** ((2 * idx) / dim))
    angles = t[:, None] * w[None, :]
    sin = torch.sin(angles)
    cos = torch.cos(angles)
    return torch.cat([sin, cos], dim=-1)


def label_embedding(x, dim):
    pass

### Собственно какие идеи у меня?

Мне очень нравится идеи с тем, что аддитивно добавляя время картинка перемещается в свою область. Очевидно, я буду отображать метку в какой то вектор, а после добавлять также аддитивно (хотя можно и мультипликативно), но есть мысль, что если два раза пройтись ReLu(Conv(Relu(Conv(x)) + t)), то может быть проблема того, что информация просто потеряется и хотелось бы научиться это делать параллельно

Пусть f: t $\rightarrow$ v отображение времени в вектор. Есть крутая идея того, что все векторы при t $\in$ [0, 1000] f(t) не образует полноценный базис и метку отобразить в ортогональное дополнение и передвигать в тэтом направлении но это слишком сложно

### Идея 1

Можно попробовать просто добавить один слой

y - метка

h = ReLU(Conv(x)) \
h = ReLU(Conv(h + y)) \
h = ReLU(Conv(h + t))

In [13]:
class Unet1(nn.Module):
    class Block(nn.Module):
        def __init__(self, in_ch, out_ch, v_dim):
            super().__init__()
            self.conv1 = nn.Conv2d(in_ch, out_ch, kernel_size=3, padding=1)
            self.conv2 = nn.Conv2d(out_ch, out_ch, kernel_size=3, padding=1)
            self.conv3 = nn.Conv2d(out_ch, out_ch, kernel_size=3, padding=1)
            self.mlp = nn.Linear(v_dim, out_ch)
            self.relu = nn.ReLU()

        def forward(self, x, time_embedding, label_embedding):
            v1 = self.mlp(label_embedding).view(time_embedding.shape[0], -1, 1, 1)
            v2 = self.mlp(time_embedding).view(time_embedding.shape[0], -1, 1, 1)
            
            h = self.relu(self.conv1(x))
            h = self.relu(self.conv2(h + v1))
            h = self.relu(self.conv3(h + v2))

            if x.shape[1] == h.shape[1]:
                return x + h
            return h

    def __init__(self, v_dim):
        super().__init__()
        self.v_dim = v_dim
        self.init_conv = nn.Conv2d(1, 64, kernel_size=3, padding=1)

        self.down1 = Unet1.Block(64, 64, v_dim)
        self.to_128 = nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1)

        self.down2 = Unet1.Block(128, 128, v_dim)
        self.to_256 = nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1)

        self.bottleneck = Unet1.Block(256, 256, v_dim)
        
        self.up_to_14 = nn.Upsample(scale_factor=2, mode='bilinear')
        self.up_block1 = Unet1.Block(384, 128, v_dim)

        self.up_to_28 = nn.Upsample(scale_factor=2, mode='bilinear')
        self.up_block2 = Unet1.Block(192, 64, v_dim)

        self.final_conv = nn.Conv2d(64, 1, kernel_size=1)

    def forward(self, x, t, y):
        t_emb = time_embedding(t, self.v_dim)
        y_emb = label_embedding(y, self.v_dim)

        x = self.init_conv(x)

        s1 = self.down1(x, t_emb, y_emb)
        x = self.to_128(s1)

        s2 = self.down2(x, t_emb, y_emb)
        x = self.to_256(s2)

        x = self.bottleneck(x, t_emb, y_emb)

        x = self.up_to_14(x)
        x = torch.cat([x, s2], dim=1)
        x = self.up_block1(x, t_emb, y_emb)

        x = self.up_to_28(x)
        x = torch.cat([x, s1], dim=1)
        x = self.up_block2(x, t_emb, y_emb)

        return self.final_conv(x)

In [14]:
%%time

device = "cuda" if torch.cuda.is_available() else "cpu"
time_dim=128
model = Unet1(time_dim).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=2e-4)

epochs = 20

alphas_cumprod_tensor = alphas_cumprod.to(device)

for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    for batch in train_loader:
        x0 = batch[0].to(device)
        t = torch.randint(0, timesteps, (x0.shape[0],), device=device)
        real_noise = torch.randn_like(x0).to(device)
        
        xt = q_sample(x0, t, alphas_cumprod_tensor, real_noise)

        predicted_noise = model(xt, t, )
        loss = torch.nn.functional.mse_loss(predicted_noise, real_noise)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    torch.cuda.synchronize()
    print(f'Эпоха {epoch}: Loss MSE: {epoch_loss / len(train_loader)}')

CPU times: total: 3 s
Wall time: 5.15 s


NameError: name 'y' is not defined

### Идея 2

Изначально функция f является отображнием f: t $\rightarrow$ v. Можно представить это как отображение от 2 скаляров f: (y, t) $\rightarrow$ v

In [ ]:
class Unet2(nn.Module):
    pass